In [1]:
#Dependencies 
import os
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix

**Import data**

In [3]:
#Import data in csv file as DataFrame
MathE_data = pd.read_csv('transformed_MathE_data.csv')
MathE_data_np = MathE_data.select_dtypes(include=[np.number])
# print(MathE_data_np[MathE_data_np > np.finfo(np.float64).max])
# print(MathE_data_np[np.isinf(MathE_data_np)].dropna(how='all'))

In [162]:
#Split entire dataset with 80/20 train-to-test set ratio
from sklearn.model_selection import train_test_split
strat_train_set, strat_test_set = train_test_split(MathE_data, stratify=MathE_data["Question Level"], test_size = 0.2, random_state=42)
MathE = strat_test_set.copy()

**Split data into 10 train/test sets with 20% of the data allocated to testing and Stratify dataset based on question level attributes**

In [ ]:
# #from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
#stratifying data by question level (Basic/Advanced)
# splitter = StratifiedShuffleSplit(n_splits= 10, train_size=0.8, random_state=42)
# stratified_splits = []
# for train_index, test_index in splitter.split(MathE_data,MathE_data["Question Level"]):
#     train_set = MathE_data.iloc[train_index]
#     test_set = MathE_data.iloc[test_index]
#     stratified_splits.append([train_set,test_set])
# print(f"Startified dataset{stratified_splits[0][0]['Question Level'].value_counts()/len(stratified_splits[0][0]['Question Level'])}/t",f"Full dataset:{MathE_data['Question Level'].value_counts()/len(MathE_data['Question Level'])}")

In [ ]:
#Separate training set into model's inputs and labels
# MathE_data_input = stratified_splits[0][0].drop(["Student ID"],axis=1)
# MathE_data_labels = stratified_splits[0][1]["Type of Answer"]
# MathE_data_input,MathE_data_labels

In [136]:
#features segregation
get_rid_of_features = ["Type of Answer","Question Level","Topic","Subtopic","Student Country"]

In [138]:
#Split entire dataset with 80/20 train-to-test set ratio
from sklearn.model_selection import train_test_split
strat_train_set, strat_test_set = train_test_split(MathE_data, stratify=MathE_data["Question Level"], test_size = 0.2, random_state=42)
MathE_train_set_labels = strat_train_set["Type of Answer"]
MathE_train_set_input = strat_train_set.copy().drop(get_rid_of_features, axis=1)

**Visualize data to gain further insight**

**Encoding text Attributes**

In [ ]:
#Encoding the "Type of Answer" Attributes as sparse matrices
# from sklearn.preprocessing import OneHotEncoder
# one_hot_encoder = OneHotEncoder()
# Encoded_Data = one_hot_encoder.fit_transform(MathE.select_dtypes(include=object))
# Encoded_Data.toarray()
# one_hot_encoder.get_feature_names_out()

In [ ]:
#plot Question ID on a histogram
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12,8))
# MathE_data_input_num["Question ID"].hist()
# plt.title('Question ID frequency')
# plt.xlabel('Question ID')
# plt.ylabel('Count')
#Note that Question ID is heavy-tailed thus prior to scaling this attribute through normalization, the log of this attribute should be calculated

**Using custom transformer to apply a logarithmic transformation of the data**

In [ ]:
# #log transformation of Question ID
# from sklearn.preprocessing import FunctionTransformer
# log_transformer = FunctionTransformer(np.log) 
# log_Question_ID = log_transformer.transform(MathE_data[["Question ID"]])
# log_Question_ID

In [ ]:
# #plot log of Question ID 
# plt.figure(figsize=(12,8))
# log_Question_ID.hist()
# plt.title('log Question ID frequency')
# plt.xlabel('log Question ID')
# plt.ylabel('Count')

**Scaling data by normalizing it**

In [ ]:
#Min-Max scaling and Normalization 
# from sklearn.preprocessing import MinMaxScaler
# Scaler = MinMaxScaler(feature_range=(-1,1))
# Normalized_Question_ID = pd.DataFrame(Scaler.fit_transform(MathE_data[["Question ID"]]),columns =['Normalized log Question ID'])
# Normalized_Question_ID

In [ ]:
#plot log of Normalized Question ID 
# plt.figure(figsize=(12,8))
# Normalized_Question_ID.hist()
# plt.title('Normalized log Question ID frequency')
# plt.xlabel('Normalized log Question ID')
# plt.ylabel('Count')

**The purpose of this section is to build a preprocessing data pipeline that perform the following tasks:**
- Transform numerical attributes by calculating the log of the attributes and scaling them
- Transform categorical attributes by OneHote encoding the attribute
- Note that we will ignore missing values
- estimator to use:
- make_pipeline()

In [100]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder

#
#Create a custom transformer that calculate the  log of a numerical 
log_transformer = FunctionTransformer(np.log)

#Create a pipeline that transform numerical attributes
numeric_pipeline = make_pipeline(MinMaxScaler(feature_range=(-1,1)))

#Create a pipeline that encode categorical attributes into sparse matrices
category_pipeline = make_pipeline(OneHotEncoder())

#Wrap both pipeline into a column transformer that automatically apply respective pipelines to both categorical and numeric attributes
preprocessing = make_column_transformer((numeric_pipeline, make_column_selector(dtype_include= np.number)),(category_pipeline, make_column_selector(dtype_include=object)))

In [ ]:
#Fit and transform "MathE" dataset
#Transformed_MathE = preprocessing.fit_transform(MathE_train_set_input).toarray()

In [ ]:
#processed_mathe = pd.DataFrame(Transformed_MathE, columns =preprocessing.get_feature_names_out(), index=MathE_train_set_input.index) 

In [ ]:
#processed_mathe

In [102]:
#Train linear model
from sklearn.linear_model import LinearRegression
Fit_model = make_pipeline(preprocessing,LinearRegression())
Fit_model.fit(MathE_train_set_input,MathE_train_set_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler(feature_range=(-1,
                                                                                               1)))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65E90>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65B10>)])),
                ('linearregression', LinearRegression())])

In [164]:
#Linear model: make a prediction
Linear_model_prediction = Fit_model.predict(MathE_train_set_input)

In [166]:
#Linear model: calculate RMSE  
from sklearn.metrics import mean_squared_error
Linear_model_RMSE= mean_squared_error(MathE_train_set_labels,Linear_model_prediction, squared=False)

In [108]:
#DescisionTreeRegression: train model, predict
from sklearn.tree import DecisionTreeRegressor
Fit_model_DTR = make_pipeline(preprocessing,DecisionTreeRegressor(random_state=42))
Fit_model_DTR.fit(MathE_train_set_input,MathE_train_set_labels)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler(feature_range=(-1,
                                                                                               1)))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65E90>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65B10>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [110]:
DTR_model_prediction = Fit_model_DTR.predict(MathE_train_set_input)

In [168]:
#Decsion Tree Regression model: calculate RMSE  
DTR_model_RMSE = mean_squared_error(MathE_train_set_labels,DTR_model_prediction, squared=False)


In [114]:
#Random Forest Reagression (RFR) model: preprocessing, train model, predict, RMSE calculation
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
Fit_model_RFR = make_pipeline(preprocessing,RandomForestRegressor(random_state=42))
Fit_model_RFR.fit(MathE_train_set_input,MathE_train_set_labels)
RFR_model_prediction = Fit_model_RFR.predict(MathE_train_set_input)

In [170]:
RFR_model_RMSE = mean_squared_error(MathE_train_set_labels,RFR_model_prediction, squared=False)

In [ ]:
#Histogram Gradient Boosting (HGB) model: preprocessing, train model, predict, RMSE calculation
#Fit_model_HGB = make_pipeline(preprocessing, HistGradientBoostingRegressor(random_state=42))
#Fit_model_HGB.fit(MathE_train_set_input, MathE_train_set_labels)
#HGB_model_prediction = Fit_model_HGB.predict(MathE_train_set_input)

In [ ]:
#HGB_model_RMSE = mean_squared_error(MathE_train_set_labels,HGB_model_pr

In [118]:
def Binary_output(dataset,parameter=0.5, set_column_name="Transformed output"):
    #This function assigned a value of either 0 or 1 depending on whether the datum is below or equal to a set hyperparameter
    #expected input is  1 x n table or a column
    transformed_dataset=[]
    for instance in dataset:
        if instance <= parameter:
            transformed_dataset.append(0)
        else:
            transformed_dataset.append(1)
    return pd.DataFrame(transformed_dataset, columns=[set_column_name], index =list(range(0,len(dataset))))

def RMSE_Binary_output(dataset,labels,parameter=0.5):
    prediction = Binary_output(dataset,parameter)
    return mean_squared_error(labels,prediction,squared=False)



In [120]:
Prediction_summary = []
predictions = {"Linear_model_prediction":Linear_model_prediction,"DTR_model_prediction":DTR_model_prediction,"RFR_model_prediction":RFR_model_prediction}
for predict in predictions:
    Prediction_summary.append(RMSE_Binary_output(predictions[predict],MathE_train_set_labels,0.5))
Prediction_summary

[0.6822220496961903, 0.28927937415620447, 0.28927937415620447]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(Df["Hyperparameter"],Df["DTR RMSE"], c="yellow",marker="^",edgecolors="green",s=200,linewidths=2,label='DescisionTreeRegression')
plt.scatter(Df["Hyperparameter"],Df["LR RMSE"], c="blue",marker="^",edgecolors="pink",s=200,linewidths=2,label='Linear Regression')
plt.xlabel("Hyperparameter")
plt.ylabel("RMSE")
plt.legend()
plt.show()

**The data shows that DTR model performs better on the training set than both the Random Forest Regressor and the linear regression model**

**The DTR RMSE reaches its minimum when the Binary split parameter is apprximately 0.5**

**thes RMSE is near .3 there is no appearant indication that the model has overfitted the training set**

---------------------------------------------------------------------------------------------------

**Evaluating the model using cross-validation**

In [122]:
#performing the cross-validation without the the hyperparameter optimization
from sklearn.model_selection import cross_val_score
Cross_Eval = -cross_val_score(Fit_model_RFR, MathE_train_set_input, MathE_train_set_labels, scoring="neg_root_mean_squared_error",cv=10)

In [124]:
pd.Series(Cross_Eval).describe().round(2)

count    10.00
mean      0.51
std       0.01
min       0.50
25%       0.51
50%       0.51
75%       0.52
max       0.52
dtype: float64

**The model performing poorly after cross-validation. This could be an indication that the model was overfitting the data initially??**

**It perform worse than the linear model**

**Using RandomSearchCV to find the optimal hyperparameter for our model** 

In [126]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distributions = {
    'randomforestregressor__max_features': randint(low=2, high=20),
}
Random_search = RandomizedSearchCV(Fit_model_RFR,param_distributions,n_iter=10, cv=3,scoring="neg_root_mean_squared_error",random_state=42)

In [128]:
Random_search.fit(MathE_train_set_input,MathE_train_set_labels)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('minmaxscaler',
                                                                                                MinMaxScaler(feature_range=(-1,
                                                                                                                            1)))]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65E90>),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                OneHotEncoder())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000002108AC65B10>)])),
                                             ('randomforestregressor',
                                              RandomForestRegressor(random_state=42))]),
                   param_distributions={'randomforestregressor__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002108A9220D0>},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [130]:
best_model = Random_search.best_estimator_

In [132]:
feature_importance = best_model["randomforestregressor"].feature_importances_
sorted(zip(feature_importance,best_model["columntransformer"].get_feature_names_out()),reverse=True)

[(0.6331860603632784, 'pipeline-1__Question ID'),
 (0.3668139396367216, 'pipeline-1__Student ID')]

**The most optimal hyperparmater for _max_features is 16**

**The 2 best predictors are Question ID and Student ID**

**Evaluate model on test set**

In [141]:
MathE_test_set_labels = strat_test_set["Type of Answer"]
MathE_test_set_input = strat_test_set.copy().drop(get_rid_of_features, axis=1)

In [147]:
final_prediction = best_model.predict(MathE_test_set_input)
final_rmse = mean_squared_error(MathE_test_set_labels,final_prediction, squared=False)
print(final_rmse.round(2))

0.51


In [151]:
#Calculate the confidence interval range of the error
from scipy import stats
confidence = 0.95
squared_errors = (final_prediction-MathE_test_set_labels)**2
np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,loc=squared_errors.mean(),scale=stats.sem(squared_errors))).round(2)

array([0.5 , 0.53])

**Save the model**

In [159]:
import joblib
joblib.dump(best_model,"RFR_MathE_answer_model.pkl")

['RFR_MathE_answer_model.pkl']